# Импорт необходимых библиотек

In [29]:
import os
import docx
import re
import pandas as pd 
import numpy as np
import pickle

import matplotlib.pyplot as plt 
import seaborn as sns

from collections import Counter

from sklearn import metrics
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer as snow



# Загрузка модели

In [30]:
# Загрузка модели 
def get_model(path_model):
    with open(path_model, 'rb') as file:
        pickle_model = pickle.load(file)
    return pickle_model


In [31]:
# Поставить местоположение у тебя
path_model = r"model\LinearSVC_model_2700.pkl"
model = get_model(path_model)


# Загрузка файла

In [32]:
def load_file(path_to_file):
    doc = docx.Document(path_to_file)
    all_paragraphs = doc.paragraphs
    list_text_doc = [par.text for par in all_paragraphs]
    len_doc_paragraph = len(all_paragraphs)
    return list_text_doc, len_doc_paragraph

In [33]:
path_document = r"C:\Users\79219\Desktop\digital_xakaton\document\Postanovlenie_Pravitelstva_Rossiyskoy_Federatsii_ot_04_03_2021__314.docx"
list_text_doc, len_paragraph = load_file(path_document)

In [34]:
def load_file(path_to_file):
    doc = docx.Document(path_to_file)
    all_paragraphs = doc.paragraphs
    list_text_doc = [par.text for par in all_paragraphs]
    len_doc_paragraph = len(all_paragraphs)
    return list_text_doc, len_doc_paragraph

In [35]:
list_text_doc, len_paragraph = load_file(path_document)

In [36]:
def pandas_df_text(text_doc):
    df = pd.DataFrame(text_doc, columns=['text'])
    return df

In [37]:
df = pandas_df_text(list_text_doc)

# Обработка текста 

In [38]:
stop_words = set(stopwords.words('russian'))
stemmer = snow('russian')
text_cleaning_re = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"

def clear_text(text):
    tokens = []
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
    text = re.sub(r'\s+', ' ', text)
    for token in text.split():
        if token not in stop_words:
            token = "".join(c for c in token if token.isalnum())
            tokens.append(token)
    return " ".join(tokens)

def pre_process(text):
    tokens = []
    for token in text.split():
        tokens.append(stemmer.stem(token))
    return " ".join(tokens)


In [39]:
def preprocess_text(df_text_doc):
    texts_ = df_text_doc.text.apply(lambda x: clear_text(x))
    texts_stemm = texts_.apply(lambda x: pre_process(x))
    return  texts_stemm

In [40]:
clear_stem_text = preprocess_text(df)

# Создание TFiDF

In [41]:
tfidf_doc=TfidfVectorizer(max_features=2745,ngram_range=(1,4))

# Создание переменной X


In [42]:
X = tfidf_doc.fit_transform(clear_stem_text)

# Ответ модели 

In [43]:
Ypredict = model.predict(X)
Ypredict = Ypredict.tolist()

# Создание списка таргетов

In [44]:
range_ = [i for i in range(len(Ypredict))]
dict_={}
for i in range_:
    dict_[i] = Ypredict[i]

# Сортировка таргетов, которые присутствуют в документе 

In [45]:
class_ = [str(i) for i in range(1,41)]
class_[-1] = 'non'

def get_info_class_loss(Ypredict, class_):
    list_class = []
    for i in Ypredict:
        if i not in list_class:
            list_class.append(i)
    return sorted(list_class)
        

In [46]:
x = get_info_class_loss(Ypredict, class_)

# Количество классов в документе

In [47]:
cnt = Counter(Ypredict)
cnt.most_common()

[('non', 54),
 ('11', 12),
 ('35', 8),
 ('10', 7),
 ('22', 3),
 ('18', 2),
 ('19', 2),
 ('12', 2),
 ('15', 2),
 ('24', 2),
 ('38', 2),
 ('20', 1),
 ('27', 1),
 ('36', 1),
 ('30', 1),
 ('13', 1)]